In [ ]:
import os
import csv
import pandas as pd
import numpy as np
#
import collections
from collections import defaultdict, OrderedDict
import queue as Q
#
import matplotlib.pyplot as plt
#

In [ ]:
# Definimos algunas funciones

In [ ]:
def compute_at_backend(q):
    return query_costs[q]

##### Las siguientes funciones de gestion de un cache (con una política) son demostrativas y no están impementadas con criterios de eficiencia.

In [ ]:
class LRUCache:
    def __init__(self, size):
        self.size = size
        self.lru_cache = collections.OrderedDict()
    def get(self, key):
        try:
            value = self.lru_cache.pop(key)
            self.lru_cache[key] = value
            return value
        except KeyError:
            return -1
    def put(self, key, value):
        try:
            self.lru_cache.pop(key)
        except KeyError:
            if len(self.lru_cache) >= self.size:
                self.lru_cache.popitem(last=False)
            self.lru_cache[key] = value
    def dump_cache(self):
        print (self.lru_cache)

In [ ]:
class LFUCache:
    def __init__(self, capacity):
        self.remain = capacity
        self.least_freq = 1
        self.node_for_freq = collections.defaultdict(collections.OrderedDict)
        self.node_for_key = dict()
    def _update(self, key, value):
        _, freq = self.node_for_key[key]
        self.node_for_freq[freq].pop(key)
        if len(self.node_for_freq[self.least_freq]) == 0:
            self.least_freq += 1
        self.node_for_freq[freq+1][key] = (value, freq+1)
        self.node_for_key[key] = (value, freq+1)
    def get(self, key):
        if key not in self.node_for_key:
            return -1
        value = self.node_for_key[key][0]
        self._update(key, value)
        return value
    def put(self, key, value):
        if key in self.node_for_key:
            self._update(key, value)
        else:
            self.node_for_key[key] = (value,1)
            self.node_for_freq[1][key] = (value,1)
            if self.remain == 0:
                try:
                    removed = self.node_for_freq[self.least_freq].popitem(last=False)
                    self.node_for_key.pop(removed[0])
                except:
                    pass
            else:
                self.remain -= 1
                self.least_freq = 1
                #
    def dump(self):
        print (self.node_for_freq)

In [ ]:
class FxCCache:
    def __init__(self, capacity):
        self.remain = capacity
        self.item_fxcq  = Q.PriorityQueue()     # (fxc, key)
        self.item_valid = dict()                # Para una key, guarda el fxc válido
        self.item_freq  = dict()                # Frecuencia de key
        self.item_cost  = dict()                # Costo de key
    #
    def dump_freq(self):
        print (self.item_freq)
    #
    def dump_queue(self):
        tmp = Q.PriorityQueue() # (fxc, key)
        while not self.item_fxcq.empty():
            it = self.item_fxcq.get()
            print (it, end = ' '),
            tmp.put(it)
        print ()
        self.item_fxcq = tmp
    #
    def _update(self, key):
        self.item_freq[key] += 1
        fxc = (self.item_freq[key]) * self.item_cost[key]
        self.item_valid[key] = fxc
        self.item_fxcq.put((fxc, key))
        #print (fxc)
        return fxc
    
    def get(self, key):
        if key not in self.item_freq:
            return -1
        up = self._update(key)       # Tengo que freq+=1 y actualizar fx en la cola
        return up

    #
    def put(self, key, cost):
        if key in self.item_freq:
            self._update(key) # Tengo que freq+=1 y actualizar fxc en la cola
        else:
            self.item_freq[key] = 1
            self.item_cost[key] = cost
            fxc = cost
            #
            if self.remain == 0:
                while True:
                    remove = self.item_fxcq.get()
                    rkey = remove[1]
                    valid_fxc = self.item_valid[rkey]
                    if remove[0] == valid_fxc:
                        try:
                            del(self.item_freq[rkey])
                            del(self.item_cost[rkey])
                        except:
                            pass
                        break
            else:
                self.remain -= 1
            # El item lo inserto despues de remover el menor
            self.item_fxcq.put((fxc, key))
            self.item_valid[key] = fxc

---
### MAIN
---

In [ ]:
# Se carga en un diccionario el costo de los queries del log para simular el tiempo de cómputo en el 
# back end. Este diccionario es utilizado por la función 'compute_at_backend(q)'
query_costs = {}
with open("../data/AOL_360k_sample.unique.times", "r") as fin:
    for line in fin:
        q, t = line.strip().split(",")
        query_costs[q] = float(t)

In [ ]:
# Se lee el archivo de consultas (completo) y se generan dos listas con los queries y costos, 
# respectivamente. Éstas se van a usar para la simulación de la performance de las politicas 
# de cache y para calcular estadísticas básicas.queries_to_process = []
queries_to_process = []
times_to_process = []
#
with open("../data/AOL_360k_sample.txt", "r") as fin:
    for query in fin:
        query = query.strip()
        cost = compute_at_backend(query)
        #
        queries_to_process.append(query)
        times_to_process.append(cost)
    #
total_queries = len(queries_to_process)

In [ ]:
# Graficamos los costos de los queries
y_axis = sorted(times_to_process)
x_axis = range(0, len(y_axis))
#x_axis = [x / total_queries for x in x_axis]   # Eje-x como proporción del total
#
plt.scatter(x_axis, y_axis)
plt.grid()
plt.xlabel("Query #")
plt.ylabel("Cost (ms)")
plt.show()

In [ ]:
# Estadísticas de costos del set de queries
total_time = sum(times_to_process)
mean_time  = np.mean(times_to_process)
p90_time   = np.percentile(times_to_process, 90)
p99_time   = np.percentile(times_to_process, 99)
max_time   = max(times_to_process)

In [ ]:
print (f'Cantidad de queries: {total_queries}')
print (f'Tiempo total       : {total_time:.2f} ms')
print (f'Tiempo medio       : {mean_time:.2f} ms')
print (f'Tiempo P90 (tail)  : {p90_time:.2f} ms')
print (f'Tiempo P99 (tail)  : {p99_time:.2f} ms')
print (f'Tiempo Máximo P100): {max_time:.2f} ms')

**Tarea 1: Probar LRU con el ejemplo de las diapos**

In [ ]:
mycache = LRUCache(3)

In [ ]:
mycache.put(1, "A")
mycache.put(2, "B")
mycache.put(3, "C")
mycache.dump_cache()

In [ ]:
mycache.get(2)
mycache.dump_cache()

In [ ]:
mycache.get(4)

In [ ]:
mycache.put(4, "D")
mycache.dump_cache()

In [ ]:
mycache.get(3)
mycache.dump_cache()

In [ ]:
mycache.put(1, "A")
mycache.dump_cache()

In [ ]:
mycache.get(3)
mycache.dump_cache()

#### Hit Rate

**Tarea 2: Evaluar Hit rate para diferentes tamaños de cache**

In [ ]:
cache_sizes = [1000, 5000, 10000, 25000, 50000, 75000, 100000]

In [ ]:
# Itero sobre los tamaños de cache a evaluar
LRU_hit_ratios = []
for csize in cache_sizes:
    mycache = LRUCache(csize)
    #
    hits   = 0
    misses = 0
    for query in queries_to_process:
        if mycache.get(query) != -1:
            hits+=1
        else:
            mycache.put(query, "dummy")
            misses+=1
    hit_ratio = hits / (hits+misses)
    LRU_hit_ratios.append(hit_ratio)
    print (f'Cache size {csize}, hits: {hits}, misses: {misses}, Hit ratio: {hit_ratio:.3f}')

In [ ]:
# Itero sobre los tamaños de cache a evaluar
LFU_hit_ratios = []
for csize in cache_sizes:
    mycache = LFUCache(csize)
    #
    hits   = 0
    misses = 0
    for query in queries_to_process:
        if mycache.get(query) != -1:
            hits+=1
        else:
            mycache.put(query, "dummy")
            misses+=1
    hit_ratio = hits / (hits+misses)
    LFU_hit_ratios.append(hit_ratio)
    print (f'Cache size {csize}, hits: {hits}, misses: {misses}, Hit ratio: {hit_ratio:.3f}')

In [ ]:
# Itero sobre los tamaños de cache a evaluar
FxC_hit_ratios = []
for csize in cache_sizes:
    mycache = FxCCache(csize)
    #
    hits   = 0
    misses = 0
    for query in queries_to_process:
        if mycache.get(query) != -1:
            hits+=1
        else:
            mycache.put(query, compute_at_backend(query))
            misses+=1
    hit_ratio = hits / (hits+misses)
    FxC_hit_ratios.append(hit_ratio)
    print (f'Cache size {csize}, hits: {hits}, misses: {misses}, Hit ratio: {hit_ratio:.3f}')

In [ ]:
# Plot

In [ ]:
plt.plot(cache_sizes, LRU_hit_ratios, 'o--', markersize=6, label="LRU")
plt.plot(cache_sizes, LFU_hit_ratios, 'x--', markersize=6, label="LFU")
plt.plot(cache_sizes, FxC_hit_ratios, '^--', markersize=6, label="FxC")
#
plt.grid()
plt.xlabel("Cache size")
plt.ylabel("Hit rate")
#
locs, labels = plt.xticks(cache_sizes, rotation=90)
plt.legend(loc=4)
plt.show()

**Tarea 3: Evaluar costo total para diferentes tamaños de cache**

In [ ]:
# Itero sobre los tamaños de cache a evaluar
LRU_total_costs = []
for csize in cache_sizes:
    mycache = LRUCache(csize)
    #
    hits   = 0
    misses = 0
    #
    cost   = 0
    total_cost   = 0
    #
    for query in queries_to_process:
        if mycache.get(query) != -1:
            cost = 0
            hits+=1
        else:
            cost = compute_at_backend(query)
            mycache.put(query, "dummy")
            misses+=1
        total_cost += cost    
    LRU_total_costs.append(total_cost)
    print (f'Cache size {csize}, hits: {hits}, misses: {misses}, Total cost: {total_cost:.3f}')

In [ ]:
# Itero sobre los tamaños de cache a evaluar
LFU_total_costs = []
for csize in cache_sizes:
    mycache = LFUCache(csize)
    #
    hits   = 0
    misses = 0
    #
    cost   = 0
    total_cost   = 0
    #
    for query in queries_to_process:
        if mycache.get(query) != -1:
            cost = 0
            hits+=1
        else:
            cost = compute_at_backend(query)
            mycache.put(query, "dummy")
            misses+=1
        total_cost += cost    
    LFU_total_costs.append(total_cost)
    print (f'Cache size {csize}, hits: {hits}, misses: {misses}, Total cost: {total_cost:.3f}')

In [ ]:
# Itero sobre los tamaños de cache a evaluar
FxC_total_costs = []
for csize in cache_sizes:
    mycache = FxCCache(csize)
    #
    hits   = 0
    misses = 0
    #
    cost   = 0
    total_cost   = 0
    #
    for query in queries_to_process:
        if mycache.get(query) != -1:
            cost = 0
            hits+=1
        else:
            cost = compute_at_backend(query)
            mycache.put(query, cost)
            misses+=1
        total_cost += cost    
    FxC_total_costs.append(total_cost)
    print (f'Cache size {csize}, hits: {hits}, misses: {misses}, Total cost: {total_cost:.3f}')

In [ ]:
plt.plot(cache_sizes, LRU_total_costs, 'o--', markersize=6, label="LRU")
plt.plot(cache_sizes, LFU_total_costs, 'x--', markersize=6, label="LFU")
plt.plot(cache_sizes, FxC_total_costs, '^--', markersize=6, label="FxC")
#
plt.grid()
plt.xlabel("Cache size")
plt.ylabel("Total cost")
#
locs, labels = plt.xticks(cache_sizes, rotation=90)
plt.legend(loc=1)
plt.show()

**Tarea 4: Calcular Ahorro entre FxC y LFU**

In [ ]:
# ToDo

**Evaluo LRU con WarmUp**

In [ ]:
# Itero sobre los tamaños de cache a evaluar
LRU_wu_hit_ratios = []
for csize in cache_sizes:
    mycache = LRUCache(csize)
    #
    warmup_queries = csize   
    #
    hits   = 0
    misses = 0
    for i, query in enumerate(queries_to_process):
        if mycache.get(query) != -1:
            if i > warmup_queries:
                hits+=1
        else:
            mycache.put(query, "dummy")
            if i > warmup_queries:
                misses+=1
    hit_ratio = hits / (hits+misses)
    LRU_wu_hit_ratios.append(hit_ratio)
    print (f'Cache size {csize}, hits: {hits}, misses: {misses}, Hit ratio: {hit_ratio:.3f}')

In [ ]:
plt.plot(cache_sizes, LRU_hit_ratios, 'o--', markersize=6, label="LRU")
plt.plot(cache_sizes, LRU_wu_hit_ratios, 'x--', markersize=6, label="LRU w/WarmUp")
#
plt.grid()
plt.xlabel("Cache size")
plt.ylabel("Hit rate")
#
locs, labels = plt.xticks(cache_sizes, rotation=90)
plt.legend(loc=4)
plt.show()

**Tarea 5: Evaluar LRU con una política de admisión basada en singletons**

In [ ]:
singletons = {}
with open("../data/AOL_360k_sample.singletons", "r") as fin:
    for sing in fin:
        sing = sing.strip()
        singletons[sing] = 1
    #
print (f'Se leyeron {len(singletons)} singletons')

In [ ]:
# Itero sobre los tamaños de cache a evaluar
LRU_nosing_hit_ratios = []
for csize in cache_sizes:
    mycache = LRUCache(csize)
    #
    hits   = 0
    misses = 0
    for query in queries_to_process:
        if mycache.get(query) != -1:
            hits+=1
        else:
            if query not in singletons:
                mycache.put(query, "dummy")
            misses+=1
    hit_ratio = hits / (hits+misses)
    LRU_nosing_hit_ratios.append(hit_ratio)
    print (f'Cache size {csize}, hits: {hits}, misses: {misses}, Hit ratio: {hit_ratio:.3f}')

In [ ]:
plt.plot(cache_sizes, LRU_hit_ratios, 'o--', markersize=6, label="LRU")
plt.plot(cache_sizes, LRU_nosing_hit_ratios, 'x--', markersize=6, label="LRU nosing")
#
plt.grid()
plt.xlabel("Cache size")
plt.ylabel("Hit rate")
#
locs, labels = plt.xticks(cache_sizes, rotation=90)
plt.legend(loc=4)
plt.show()

**Pregunta: A qué se debe el comportamiento de 'LRU nosing' en la gráfica anterior?**

**Tarea 5: Evaluar LRU con WarmUp y una política de admisión basada en singletons**

In [ ]:
# ToDo